In [ ]:
import pandas as pd
import numpy as np
#import kaleido
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import holidays
from datetime import date
from dateutil.relativedelta import relativedelta 
# This code below is to import an excel or csv for python to plot
import os

# Label Peaks

In [ ]:
def clean_data(df, date_name, melt_vector = [], row  = True):
    if row:
        df = df.melt(id_vars= melt_vector, var_name='Time') 

    # Combine     
    df[date_name] = pd.to_datetime(df.Date.astype(str)+' '+df.Time.astype(str))
    # Delete time column since it is not needed
    df = df.drop(columns=['Time'])
    # Now python will sort the dataframe by time AND date
    df = df.sort_values([date_name])
    # This will renumber the resorted values from 0 to len(df)
    df = df.reset_index(drop=True)
    # This will rename the column  
    df = df.rename(columns={'value': 'Demand'})
    # This will convert everything to numeric
    df.Demand = pd.to_numeric(df.Demand, errors='coerce')
    
    return df

In [ ]:
 # -*- coding: utf-8 -*-

#  python-holidays
#  ---------------
#  A fast, efficient Python library for generating country, province and state
#  specific sets of holidays on the fly. It aims to make determining whether a
#  specific date is a holiday as fast and flexible as possible.
#
#  Author:  ryanss <ryanssdev@icloud.com> (c) 2014-2017
#           dr-prodigy <maurizio.montel@gmail.com> (c) 2017-2020
#  Website: https://github.com/dr-prodigy/python-holidays
#  License: MIT (see LICENSE file)

from datetime import date, datetime, timedelta
from dateutil.easter import easter, EASTER_ORTHODOX
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta as rd
from dateutil.relativedelta import MO, TU, WE, TH, FR, SA, SU
import inspect
import six
import sys
import warnings

__version__ = '0.9.12'

MON, TUE, WED, THU, FRI, SAT, SUN = range(7)
WEEKEND = (SAT, SUN)

JAN, FEB, MAR, APR, MAY, JUN, JUL, AUG, SEP, OCT, \
    NOV, DEC = range(1, 13)


class HolidayBase(dict):
    PROVINCES = []

    def __init__(self, years=[], expand=True, observed=True,
                 prov=None, state=None):
        self.observed = observed
        self.expand = expand
        if isinstance(years, int):
            years = [years, ]
        self.years = set(years)
        if not getattr(self, 'prov', False):
            self.prov = prov
        self.state = state
        for year in list(self.years):
            self._populate(year)

    def __setattr__(self, key, value):
        if key == 'observed' and len(self) > 0:
            dict.__setattr__(self, key, value)
            if value is True:
                # Add (Observed) dates
                years = list(self.years)
                self.years = set()
                self.clear()
                for year in years:
                    self._populate(year)
            else:
                # Remove (Observed) dates
                for k, v in list(self.items()):
                    if v.find("Observed") >= 0:
                        del self[k]
        else:
            return dict.__setattr__(self, key, value)

    def __keytransform__(self, key):
        if isinstance(key, datetime):
            key = key.date()
        elif isinstance(key, date):
            key = key
        elif isinstance(key, int) or isinstance(key, float):
            key = datetime.utcfromtimestamp(key).date()
        elif isinstance(key, six.string_types):
            try:
                key = parse(key).date()
            except (ValueError, OverflowError):
                raise ValueError("Cannot parse date from string '%s'" % key)
        else:
            raise TypeError("Cannot convert type '%s' to date." % type(key))

        if self.expand and key.year not in self.years:
            self.years.add(key.year)
            self._populate(key.year)
        return key

    def __contains__(self, key):
        return dict.__contains__(self, self.__keytransform__(key))

    def __getitem__(self, key):
        if isinstance(key, slice):
            if not key.start or not key.stop:
                raise ValueError("Both start and stop must be given.")

            start = self.__keytransform__(key.start)
            stop = self.__keytransform__(key.stop)

            if key.step is None:
                step = 1
            elif isinstance(key.step, timedelta):
                step = key.step.days
            elif isinstance(key.step, int):
                step = key.step
            else:
                raise TypeError(
                    "Cannot convert type '%s' to int." % type(key.step)
                )

            if step == 0:
                raise ValueError('Step value must not be zero.')

            date_diff = stop - start
            if date_diff.days < 0 <= step or date_diff.days >= 0 > step:
                step *= -1

            days_in_range = []
            for delta_days in range(0, date_diff.days, step):
                day = start + timedelta(days=delta_days)
                try:
                    dict.__getitem__(
                        self,
                        day
                    )
                    days_in_range.append(day)
                except (KeyError):
                    pass
            return days_in_range
        return dict.__getitem__(self, self.__keytransform__(key))

    def __setitem__(self, key, value):
        if key in self:
            if self.get(key).find(value) < 0 \
                    and value.find(self.get(key)) < 0:
                value = "%s, %s" % (value, self.get(key))
            else:
                value = self.get(key)
        return dict.__setitem__(self, self.__keytransform__(key), value)

    def update(self, *args):
        args = list(args)
        for arg in args:
            if isinstance(arg, dict):
                for key, value in list(arg.items()):
                    self[key] = value
            elif isinstance(arg, list):
                for item in arg:
                    self[item] = "Holiday"
            else:
                self[arg] = "Holiday"

    def append(self, *args):
        return self.update(*args)

    def get(self, key, default=None):
        return dict.get(self, self.__keytransform__(key), default)

    def get_list(self, key):
        return [h for h in self.get(key, "").split(", ") if h]

    def pop(self, key, default=None):
        if default is None:
            return dict.pop(self, self.__keytransform__(key))
        return dict.pop(self, self.__keytransform__(key), default)

    def __eq__(self, other):
        return dict.__eq__(self, other) and self.__dict__ == other.__dict__

    def __ne__(self, other):
        return dict.__ne__(self, other) or self.__dict__ != other.__dict__

    def __add__(self, other):
        if isinstance(other, int) and other == 0:
            # Required to sum() list of holidays
            # sum([h1, h2]) is equivalent to (0 + h1 + h2)
            return self
        elif not isinstance(other, HolidayBase):
            raise TypeError()
        HolidaySum = createHolidaySum(self, other)
        country = (getattr(self, 'country', None) or
                   getattr(other, 'country', None))
        if self.country and other.country and self.country != other.country:
            c1 = self.country
            if not isinstance(c1, list):
                c1 = [c1]
            c2 = other.country
            if not isinstance(c2, list):
                c2 = [c2]
            country = c1 + c2
        prov = getattr(self, 'prov', None) or getattr(other, 'prov', None)
        if self.prov and other.prov and self.prov != other.prov:
            p1 = self.prov if isinstance(self.prov, list) else [self.prov]
            p2 = other.prov if isinstance(other.prov, list) else [other.prov]
            prov = p1 + p2
        return HolidaySum(years=(self.years | other.years),
                          expand=(self.expand or other.expand),
                          observed=(self.observed or other.observed),
                          country=country, prov=prov)

    def __radd__(self, other):
        return self.__add__(other)

    def _populate(self, year):
        pass


def createHolidaySum(h1, h2):
    class HolidaySum(HolidayBase):

        def __init__(self, country, **kwargs):
            self.country = country
            self.holidays = []
            if getattr(h1, 'holidays', False):
                for h in h1.holidays:
                    self.holidays.append(h)
            else:
                self.holidays.append(h1)
            if getattr(h2, 'holidays', False):
                for h in h2.holidays:
                    self.holidays.append(h)
            else:
                self.holidays.append(h2)
            HolidayBase.__init__(self, **kwargs)

        def _populate(self, year):
            for h in self.holidays[::-1]:
                h._populate(year)
                self.update(h)

    return HolidaySum


def list_supported_countries():
    """List all supported countries incl. their abbreviation."""
    return [name for name, obj in
            inspect.getmembers(sys.modules[__name__], inspect.isclass)
            if obj.__module__ is __name__]


def CountryHoliday(country, years=[], prov=None, state=None, expand=True,
                   observed=True):
    try:
        country_holiday = globals()[country](years=years,
                                             prov=prov,
                                             state=state,
                                             expand=expand,
                                             observed=observed)
    except (KeyError):
        raise KeyError("Country %s not available" % country)
    return country_holiday

 class UsElectricHolidays(holidays.HolidayBase):
 
    def _populate(self, year):
         # New Year's Day
        if year > 1870:
            name = "New Year's Day"
            self[date(year, JAN, 1)] = name
            if self.observed and date(year, JAN, 1).weekday() == SUN:
                self[date(year, JAN, 1) + rd(days=+1)] = name + \
                    " (Observed)"

        # Washington's Birthday
        name = "Washington's Birthday"
        if year > 1970:
            self[date(year, FEB, 1) + rd(weekday=MO(+3))] = name
        elif year >= 1879:
            self[date(year, FEB, 22)] = name

        # Memorial Day
        if year > 1970:
            self[date(year, MAY, 31) + rd(weekday=MO(-1))] = "Memorial Day"
        elif year >= 1888:
            self[date(year, MAY, 30)] = "Memorial Day"

        # Independence Day
        if year > 1870:
            name = "Independence Day"
            self[date(year, JUL, 4)] = name
            if self.observed and date(year, JUL, 4).weekday() == SUN:
                self[date(year, JUL, 4) + rd(days=+1)] = name + " (Observed)"

        # Labor Day
        if year >= 1894:
            self[date(year, SEP, 1) + rd(weekday=MO)] = "Labor Day"

        # Veterans Day
        if year > 1953:
            name = "Veterans Day"
        else:
            name = "Armistice Day"
        if 1978 > year > 1970:
            self[date(year, OCT, 1) + rd(weekday=MO(+4))] = name
        elif year >= 1938:
            self[date(year, NOV, 11)] = name
            if self.observed \
                    and date(year, NOV, 11).weekday() == SUN:
                self[date(year, NOV, 11) + rd(days=+1)] = name + \
                    " (Observed)"

        # Thanksgiving
        if year > 1870:
            self[date(year, NOV, 1) + rd(weekday=TH(+4))] = "Thanksgiving"
            
        # Christmas Day
        if year > 1870:
            name = "Christmas Day"
            self[date(year, DEC, 25)] = "Christmas Day"
            if self.observed \
                    and date(year, DEC, 25).weekday() == SUN:
                self[date(year, DEC, 25) + rd(days=+1)] = name + \
                    " (Observed)"
        
us_holidays = UsElectricHolidays()

In [ ]:
def label_weekends(df, timestamp_name):
    # Make the index the date
    df.index = df[timestamp_name]

    # Creates column for day of the week: 0 = Monday to 6 = Sunday
    df['DayWeek'] = df[timestamp_name].dt.dayofweek 
    # .apply run a function through every line of code without usong a for loop
    # lambda is a temporay function 
    # If x >= 5 set true (boolean statement) since it is the weekend # 5 is Saturday and 6 is Sunday
    # Returns boolean staement where weekend is true 
    df['Is_Weekend'] = df['DayWeek'].apply(lambda x: True if x >= 5 else False )
    # Creates an array for all the US Holidays in California
    # us_holidays = holidays.CountryHoliday('US')
    # Uses apply and lambda again to check if there is a holiday 
    # X in holiday returns a boolean satement 
    # .apply runs this function thorugh the entire Date column 
    # Returns a new boolean column where holidays is true 
    df['Is_Holiday'] = df[timestamp_name].apply(lambda x: True if x in us_holidays else False )
    # Creates a new column of Boolean satements of where summer is true
    # 6 is June and 9 is September
    df['is_summer'] = df[timestamp_name].dt.month.between(6,9)

    # This function determines the four catogories of Summer Weekend, Winter Weekend, Summer Weekday, Winter Weekday
    # The function takes in 4 columns in the dataframme
    # THhe .apply() function passes in one variable at a time from each column
    def peak_fun(vec):
        Date = vec[0] # First Column Date, .apply() Passes in a timestamp not a vector
        Is_Weekend = vec[1] # Second Column Is Weekend, .apply() Passes in a bool not a vector
        Is_Holiday = vec[2] # Third Column Is Holiday, .apply() Passes in a bool not a vector
        Is_Summer = vec[3] # Fourth Column Is Summer, .apply() Passes in a bool not a vector
        if Is_Weekend or Is_Holiday:
            if Is_Summer:
                return 'Summer_Weekend'       
            else:
                return 'Winter_Weekend'
        else:
            if Is_Summer:
                return 'Summer_Weekday'
            else:
                return 'Winter_Weekday'

    # Pass in the 4 columns and use the .apply() function
    df['fun'] = df[[timestamp_name, 'Is_Weekend', 'Is_Holiday', 'is_summer']].apply(peak_fun, axis = 1)
    return df 

In [ ]:
# Declare Arrays
# Summer Weekday Time
su_wdy_sofpk = {} 
su_wdy_ofpk = {} 
su_wdy_mdpk = {}
su_wdy_onpk = {}

# Summer Weekend Time
su_wkd_sofpk = {}
su_wkd_ofpk = {}
su_wkd_mdpk = {}
su_wkd_onpk = {}

# Winter Weekday Time
wt_wdy_sofpk = {}
wt_wdy_ofpk = {}
wt_wdy_mdpk = {}
wt_wdy_onpk = {}

# Winter Weekend Time
wt_wkd_sofpk = {}
wt_wkd_ofpk = {}
wt_wkd_mdpk = {}
wt_wkd_onpk = {}

In [ ]:
# RPU TOU Pricing 
# Summer Weekday Time
su_wdy_sofpk[0] = False
su_wdy_ofpk[0] = ['0:00','8:00','23:00','0:00']
su_wdy_mdpk[0] = ['8:00', '12:00', '18:00', '23:00']
su_wdy_onpk[0] = ['12:00', '18:00']

# Summer Weekend Time
su_wkd_sofpk[0] = False
su_wkd_ofpk[0] = ['0:00','21:00','21:00','0:00']
su_wkd_mdpk[0] = False
su_wkd_onpk[0] = False

# Winter Weekday Time
wt_wdy_sofpk[0] = False
wt_wdy_ofpk[0] = ['0:00','8:00','21:00','0:00']
wt_wdy_mdpk[0] = ['8:00', '17:00']
wt_wdy_onpk[0] = ['17:00', '21:00']

# Winter Weekend Time
wt_wkd_sofpk[0] = False
wt_wkd_ofpk[0] = ['0:00','21:00','21:00','0:00']
wt_wkd_mdpk[0] = False
wt_wkd_onpk[0] = False

In [ ]:
# RPU TOU Pricing 
# Summer Weekday Time
su_wdy_sofpk[1] = False
su_wdy_ofpk[1] = ['0:00','8:00','23:00','0:00']
su_wdy_mdpk[1] = ['8:00', '12:00', '18:00', '23:00']
su_wdy_onpk[1] = ['12:00', '18:00']

# Summer Weekend Time
su_wkd_sofpk[1] = False
su_wkd_ofpk[1] = ['0:00','21:00','21:00','0:00']
su_wkd_mdpk[1] = False
su_wkd_onpk[1] = False

# Winter Weekday Time
wt_wdy_sofpk[1] = False
wt_wdy_ofpk[1] = ['0:00','8:00','21:00','0:00']
wt_wdy_mdpk[1] = ['8:00', '17:00']
wt_wdy_onpk[1] = ['17:00', '21:00']

# Winter Weekend Time
wt_wkd_sofpk[1] = False
wt_wkd_ofpk[1] = ['0:00','21:00','21:00','0:00']
wt_wkd_mdpk[1] = False
wt_wkd_onpk[1] = False

In [ ]:
# RPU TOU Pricing 
# Summer Weekday Time
su_wdy_sofpk[2] = False
su_wdy_ofpk[2] = ['0:00','8:00','23:00','0:00']
su_wdy_mdpk[2] = ['8:00', '12:00', '18:00', '23:00']
su_wdy_onpk[2] = ['12:00', '18:00']

# Summer Weekend Time
su_wkd_sofpk[2] = False
su_wkd_ofpk[2] = ['0:00','21:00','21:00','0:00']
su_wkd_mdpk[2] = False
su_wkd_onpk[2] = False

# Winter Weekday Time
wt_wdy_sofpk[2] = False
wt_wdy_ofpk[2] = ['0:00','8:00','21:00','0:00']
wt_wdy_mdpk[2] = ['8:00', '17:00']
wt_wdy_onpk[2] = ['17:00', '21:00']

# Winter Weekend Time
wt_wkd_sofpk[2] = False
wt_wkd_ofpk[2] = ['0:00','21:00','21:00','0:00']
wt_wkd_mdpk[2] = False
wt_wkd_onpk[2] = False

In [ ]:
# RPU TOU Pricing 
# Summer Weekday Time
su_wdy_sofpk[3] = False
su_wdy_ofpk[3] = ['0:00','8:00','23:00','0:00']
su_wdy_mdpk[3] = ['8:00', '12:00', '18:00', '23:00']
su_wdy_onpk[3] = ['12:00', '18:00']

# Summer Weekend Time
su_wkd_sofpk[3] = False
su_wkd_ofpk[3] = ['0:00','21:00','21:00','0:00']
su_wkd_mdpk[3] = False
su_wkd_onpk[3] = False

# Winter Weekday Time
wt_wdy_sofpk[3] = False
wt_wdy_ofpk[3] = ['0:00','8:00','21:00','0:00']
wt_wdy_mdpk[3] = ['8:00', '17:00']
wt_wdy_onpk[3] = ['17:00', '21:00']

# Winter Weekend Time
wt_wkd_sofpk[3] = False
wt_wkd_ofpk[3] = ['0:00','21:00','21:00','0:00']
wt_wkd_mdpk[3] = False
wt_wkd_onpk[3] = False

In [ ]:
# RPU TOU Pricing 
# Summer Weekday Time
su_wdy_sofpk[4] = False
su_wdy_ofpk[4] = ['0:00','8:00','23:00','0:00']
su_wdy_mdpk[4] = ['8:00', '12:00', '18:00', '23:00']
su_wdy_onpk[4] = ['12:00', '18:00']

# Summer Weekend Time
su_wkd_sofpk[4] = False
su_wkd_ofpk[4] = ['0:00','21:00','21:00','0:00']
su_wkd_mdpk[4] = False
su_wkd_onpk[4] = False

# Winter Weekday Time
wt_wdy_sofpk[4] = False
wt_wdy_ofpk[4] = ['0:00','8:00','21:00','0:00']
wt_wdy_mdpk[4] = ['8:00', '17:00']
wt_wdy_onpk[4] = ['17:00', '21:00']

# Winter Weekend Time
wt_wkd_sofpk[4] = False
wt_wkd_ofpk[4] = ['0:00','21:00','21:00','0:00']
wt_wkd_mdpk[4] = False
wt_wkd_onpk[4] = False

In [ ]:
# RPU TOU Pricing 
# Summer Weekday Time
su_wdy_sofpk[5] = False
su_wdy_ofpk[5] = ['0:00','8:00','23:00','0:00']
su_wdy_mdpk[5] = ['8:00', '12:00', '18:00', '23:00']
su_wdy_onpk[5] = ['12:00', '18:00']

# Summer Weekend Time
su_wkd_sofpk[5] = False
su_wkd_ofpk[5] = ['0:00','21:00','21:00','0:00']
su_wkd_mdpk[5] = False
su_wkd_onpk[5] = False

# Winter Weekday Time
wt_wdy_sofpk[5] = False
wt_wdy_ofpk[5] = ['0:00','8:00','21:00','0:00']
wt_wdy_mdpk[5] = ['8:00', '17:00']
wt_wdy_onpk[5] = ['17:00', '21:00']

# Winter Weekend Time
wt_wkd_sofpk[5] = False
wt_wkd_ofpk[5] = ['0:00','21:00','21:00','0:00']
wt_wkd_mdpk[5] = False
wt_wkd_onpk[5] = False

In [ ]:
# RPU TOU Pricing 
# Summer Weekday Time
su_wdy_sofpk[6] = False
su_wdy_ofpk[6] = ['0:00','8:00','23:00','0:00']
su_wdy_mdpk[6] = ['8:00', '12:00', '18:00', '23:00']
su_wdy_onpk[6] = ['12:00', '18:00']

# Summer Weekend Time
su_wkd_sofpk[6] = False
su_wkd_ofpk[6] = ['0:00','21:00','21:00','0:00']
su_wkd_mdpk[6] = False
su_wkd_onpk[6] = False

# Winter Weekday Time
wt_wdy_sofpk[6] = False
wt_wdy_ofpk[6] = ['0:00','8:00','21:00','0:00']
wt_wdy_mdpk[6] = ['8:00', '17:00']
wt_wdy_onpk[6] = ['17:00', '21:00']

# Winter Weekend Time
wt_wkd_sofpk[6] = False
wt_wkd_ofpk[6] = ['0:00','21:00','21:00','0:00']
wt_wkd_mdpk[6] = False
wt_wkd_onpk[6] = False

In [ ]:
# RPU TOU Pricing 
# Summer Weekday Time
su_wdy_sofpk[7] = False
su_wdy_ofpk[7] = ['0:00','8:00','23:00','0:00']
su_wdy_mdpk[7] = ['8:00', '12:00', '18:00', '23:00']
su_wdy_onpk[7] = ['12:00', '18:00']

# Summer Weekend Time
su_wkd_sofpk[7] = False
su_wkd_ofpk[7] = ['0:00','21:00','21:00','0:00']
su_wkd_mdpk[7] = False
su_wkd_onpk[7] = False

# Winter Weekday Time
wt_wdy_sofpk[7] = False
wt_wdy_ofpk[7] = ['0:00','8:00','21:00','0:00']
wt_wdy_mdpk[7] = ['8:00', '17:00']
wt_wdy_onpk[7] = ['17:00', '21:00']

# Winter Weekend Time
wt_wkd_sofpk[7] = False
wt_wkd_ofpk[7] = ['0:00','21:00','21:00','0:00']
wt_wkd_mdpk[7] = False
wt_wkd_onpk[7] = False

In [ ]:
def summer_weekday(su_wdy_sofpk, su_wdy_ofpk, su_wdy_mdpk, su_wdy_onpk, df):
    
    if su_wdy_sofpk != False:
        len_su_wdy_sofpk = int(len(su_wdy_sofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wdy_sofpk,1):
            selection = df[df['fun'] == 'Summer_Weekday'].between_time(su_wdy_sofpk[begin],su_wdy_sofpk[end], include_end=False)
            selection["Peak"] = "Super Off-Peak"
            ww = df["fun"] == "Summer_Weekday"
            df.loc[df[ww].between_time(su_wdy_sofpk[begin], su_wdy_sofpk[end], include_end=False).index, "Peak"] = "Super Off-Peak"
            begin += 2
            end += 2
            
    if su_wdy_ofpk != False:
        len_su_wdy_ofpk = int(len(su_wdy_ofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wdy_ofpk,1):
            selection = df[df['fun'] == 'Summer_Weekday'].between_time(su_wdy_ofpk[begin],su_wdy_ofpk[end], include_end=False)
            selection["Peak"] = "Off-Peak"
            ww = df["fun"] == "Summer_Weekday"
            df.loc[df[ww].between_time(su_wdy_ofpk[begin], su_wdy_ofpk[end], include_end=False).index, "Peak"] = "Off-Peak"
            begin += 2
            end += 2
    
    if su_wdy_mdpk != False:
        len_su_wdy_mdpk = int(len(su_wdy_mdpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wdy_mdpk,1):
            selection = df[df['fun'] == 'Summer_Weekday'].between_time(su_wdy_mdpk[begin],su_wdy_mdpk[end], include_end=False)
            selection["Peak"] = "Mid-Peak"
            ww = df["fun"] == "Summer_Weekday"
            df.loc[df[ww].between_time(su_wdy_mdpk[begin], su_wdy_mdpk[end], include_end=False).index, "Peak"] = "Mid-Peak"
            begin += 2
            end += 2
    
    if su_wdy_onpk != False:
        len_su_wdy_onpk = int(len(su_wdy_onpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wdy_onpk,1):
            selection = df[df['fun'] == 'Summer_Weekday'].between_time(su_wdy_onpk[begin],su_wdy_onpk[end], include_end=False)
            selection["Peak"] = "On-Peak"
            ww = df["fun"] == "Summer_Weekday"
            df.loc[df[ww].between_time(su_wdy_onpk[begin], su_wdy_onpk[end], include_end=False).index, "Peak"] = "On-Peak"
            begin += 2
            end += 2
    return df

def summer_weekend(su_wkd_sofpk, su_wkd_ofpk, su_wkd_mdpk, su_wkd_onpk, df):
    
    if su_wkd_sofpk != False:
        len_su_wkd_sofpk = int(len(su_wkd_sofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wkd_sofpk,1):
            selection = df[df['fun'] == 'Summer_Weekend'].between_time(su_wkd_sofpk[begin],su_wkd_sofpk[end], include_end=False)
            selection["Peak"] = "Super Off-Peak"
            ww = df["fun"] == "Summer_Weekend"
            df.loc[df[ww].between_time(su_wkd_sofpk[begin], su_wkd_sofpk[end], include_end=False).index, "Peak"] = "Super Off-Peak"
            begin += 2
            end += 2
            
    if su_wkd_ofpk != False:
        len_su_wkd_ofpk = int(len(su_wkd_ofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wkd_ofpk,1):
            selection = df[df['fun'] == 'Summer_Weekend'].between_time(su_wkd_ofpk[begin],su_wkd_ofpk[end], include_end=False)
            selection["Peak"] = "Off-Peak"
            ww = df["fun"] == "Summer_Weekend"
            df.loc[df[ww].between_time(su_wkd_ofpk[begin], su_wkd_ofpk[end], include_end=False).index, "Peak"] = "Off-Peak"
            begin += 2
            end += 2
    
    if su_wkd_mdpk != False:
        len_su_wkd_mdpk = int(len(su_wkd_mdpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wkd_mdpk,1):
            selection = df[df['fun'] == 'Summer_Weekend'].between_time(su_wkd_mdpk[begin],su_wkd_mdpk[end], include_end=False)
            selection["Peak"] = "Mid-Peak"
            ww = df["fun"] == "Summer_Weekend"
            df.loc[df[ww].between_time(su_wkd_mdpk[begin], su_wkd_mdpk[end], include_end=False).index, "Peak"] = "Mid-Peak"
            begin += 2
            end += 2
    
    if su_wkd_onpk != False:
        len_su_wkd_onpk = int(len(su_wkd_onpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_su_wkd_onpk,1):
            selection = df[df['fun'] == 'Summer_Weekend'].between_time(su_wkd_onpk[begin],su_wkd_onpk[end], include_end=False)
            selection["Peak"] = "On-Peak"
            ww = df["fun"] == "Summer_Weekend"
            df.loc[df[ww].between_time(su_wkd_onpk[begin], su_wkd_onpk[end], include_end=False).index, "Peak"] = "On-Peak"
            begin += 2
            end += 2
    return df

def winter_weekday(wt_wdy_sofpk, wt_wdy_ofpk, wt_wdy_mdpk, wt_wdy_onpk, df):
    
    if wt_wdy_sofpk != False:
        len_wt_wdy_sofpk = int(len(wt_wdy_sofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wdy_sofpk,1):
            selection = df[df['fun'] == 'Winter_Weekday'].between_time(wt_wdy_sofpk[begin],wt_wdy_sofpk[end], include_end=False)
            selection["Peak"] = "Super Off-Peak"
            ww = df["fun"] == "Winter_Weekday"
            df.loc[df[ww].between_time(wt_wdy_sofpk[begin], wt_wdy_sofpk[end], include_end=False).index, "Peak"] = "Super Off-Peak"
            begin += 2
            end += 2
            
    if wt_wdy_ofpk != False:
        len_wt_wdy_ofpk = int(len(wt_wdy_ofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wdy_ofpk,1):
            selection = df[df['fun'] == 'Winter_Weekday'].between_time(wt_wdy_ofpk[begin],wt_wdy_ofpk[end], include_end=False)
            selection["Peak"] = "Off-Peak"
            ww = df["fun"] == "Winter_Weekday"
            df.loc[df[ww].between_time(wt_wdy_ofpk[begin], wt_wdy_ofpk[end], include_end=False).index, "Peak"] = "Off-Peak"
            begin += 2
            end += 2
    
    if wt_wdy_mdpk != False:
        len_wt_wdy_mdpk = int(len(wt_wdy_mdpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wdy_mdpk,1):
            selection = df[df['fun'] == 'Winter_Weekday'].between_time(wt_wdy_mdpk[begin],wt_wdy_mdpk[end], include_end=False)
            selection["Peak"] = "Mid-Peak"
            ww = df["fun"] == "Winter_Weekday"
            df.loc[df[ww].between_time(wt_wdy_mdpk[begin], wt_wdy_mdpk[end], include_end=False).index, "Peak"] = "Mid-Peak"
            begin += 2
            end += 2
    
    if wt_wdy_onpk != False:
        len_wt_wdy_onpk = int(len(wt_wdy_onpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wdy_onpk,1):
            selection = df[df['fun'] == 'Winter_Weekday'].between_time(wt_wdy_onpk[begin],wt_wdy_onpk[end], include_end=False)
            selection["Peak"] = "On-Peak"
            ww = df["fun"] == "Winter_Weekday"
            df.loc[df[ww].between_time(wt_wdy_onpk[begin], wt_wdy_onpk[end], include_end=False).index, "Peak"] = "On-Peak"
            begin += 2
            end += 2
    return df

def winter_weekend(wt_wkd_sofpk, wt_wkd_ofpk, wt_wkd_mdpk, wt_wkd_onpk, df):
    
    if wt_wkd_sofpk != False:
        len_wt_wkd_sofpk = int(len(wt_wkd_sofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wkd_sofpk,1):
            selection = df[df['fun'] == 'Winter_Weekend'].between_time(wt_wkd_sofpk[begin],wt_wkd_sofpk[end], include_end=False)
            selection["Peak"] = "Super Off-Peak"
            ww = df["fun"] == "Winter_Weekend"
            df.loc[df[ww].between_time(wt_wkd_sofpk[begin], wt_wkd_sofpk[end], include_end=False).index, "Peak"] = "Super Off-Peak"
            begin += 2
            end += 2
            
    if wt_wkd_ofpk != False:
        len_wt_wkd_ofpk = int(len(wt_wkd_ofpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wkd_ofpk,1):
            selection = df[df['fun'] == 'Winter_Weekend'].between_time(wt_wkd_ofpk[begin],wt_wkd_ofpk[end], include_end=False)
            selection["Peak"] = "Off-Peak"
            ww = df["fun"] == "Winter_Weekend"
            df.loc[df[ww].between_time(wt_wkd_ofpk[begin], wt_wkd_ofpk[end], include_end=False).index, "Peak"] = "Off-Peak"
            begin += 2
            end += 2
    
    if wt_wkd_mdpk != False:
        len_wt_wkd_mdpk = int(len(wt_wkd_mdpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wkd_mdpk,1):
            selection = df[df['fun'] == 'Winter_Weekend'].between_time(wt_wkd_mdpk[begin],wt_wkd_mdpk[end], include_end=False)
            selection["Peak"] = "Mid-Peak"
            ww = df["fun"] == "Winter_Weekend"
            df.loc[df[ww].between_time(wt_wkd_mdpk[begin], wt_wkd_mdpk[end], include_end=False).index, "Peak"] = "Mid-Peak"
            begin += 2
            end += 2
    
    if wt_wkd_onpk != False:
        len_wt_wkd_onpk = int(len(wt_wkd_onpk)/2)
        begin = 0
        end = 1
        for i in range(0,len_wt_wkd_onpk,1):
            selection = df[df['fun'] == 'Winter_Weekend'].between_time(wt_wkd_onpk[begin],wt_wkd_onpk[end], include_end=False)
            selection["Peak"] = "On-Peak"
            ww = df["fun"] == "Winter_Weekend"
            df.loc[df[ww].between_time(wt_wkd_onpk[begin], wt_wkd_onpk[end], include_end=False).index, "Peak"] = "On-Peak"
            begin += 2
            end += 2
    return df

In [ ]:
def label_peaks(df, i):
    winter_weekend(wt_wkd_sofpk[i], wt_wkd_ofpk[i], wt_wkd_mdpk[i], wt_wkd_onpk[i], df)
    winter_weekday(wt_wdy_sofpk[i], wt_wdy_ofpk[i], wt_wdy_mdpk[i], wt_wdy_onpk[i], df)
    summer_weekday(su_wdy_sofpk[i], su_wdy_ofpk[i], su_wdy_mdpk[i], su_wdy_onpk[i], df)
    summer_weekend(su_wkd_sofpk[i], su_wkd_ofpk[i], su_wkd_mdpk[i], su_wkd_onpk[i], df)

In [ ]:
def resample(df, time_column = 'time', resample_rate = '15T'):
    df[time_column] = pd.to_datetime(df[time_column])
    df = df.set_index(df[time_column])
    df = df.drop(columns = [time_column])
    df = df.resample(resample_rate).mean()  #df.groupby(pd.Grouper(key=time_column, freq=resample_rate)).ffill().bfill() 
    df.insert(loc=0, column=time_column, value=df.index) #df.insert(loc=0, column=time_column, value=np.arange(0, len(df.index), 1, dtype=int)) #
    df = df.reset_index(drop=True)
    return df

In [ ]:
os.chdir('/home/lencon/Git_Research/FISTS_Data')
s1 = pd.read_hdf('scenario_0_baseline.h5', 'df')
s2 = pd.read_hdf('scenario_1_ev_chargers.h5', 'df')
s3 = pd.read_hdf('scenario_2_bess_no_ev.h5', 'df')
s4 = pd.read_hdf('scenario_3_microgrid_100_kwh.h5', 'df')
s5 = pd.read_hdf('scenario_4_microgrid_250_kwh.h5', 'df')
s6 = pd.read_hdf('scenario_5_microgrid.h5', 'df')
s7 = pd.read_hdf('scenario_6_microgrid_1000_kwh.h5', 'df')
s8 = pd.read_hdf('scenario_7_microgrid_2000_kwh.h5', 'df')
df = [s1, s2, s3, s4, s5, s6, s7, s8]
df

In [ ]:
i = 7
df[i]['time'] = pd.to_datetime(df[i]['time'] + 1646092800, unit = 's')

In [ ]:
df[0]["grid.sou.S[1]"].plot()

In [ ]:
for i in range(len(df)):
    df[i] = resample(df[i])
df

In [ ]:
for i in range(len(df)):
    df[i][df[i].select_dtypes(include=['number']).columns] /= 1000
df

Call the the function to label peaks

In [ ]:
#df = clean_data(df, "Date", ['ServiceAccount', 'Date', 'Meter Number', 'Unit of Measurement'])
for i in range(len(df)):    
    df[i] = label_weekends(df[i], 'time')
    label_peaks(df[i], i)
    df[i] = df[i].reset_index(drop = True)
df

In [ ]:
df[7]['grid.sou.S[1]'].max()

# Find Enrgy and Power Values and Costs

## Seperate interval by billing month

In [ ]:
billing = pd.read_excel("/home/lencon/Git_Research/FISTS_2024/Code/Microgrid_Data/billing_month.xlsx")
billing

In [ ]:
billMonth = billing.billing_month
offset = pd.DateOffset(minutes=15)
timestamp_name = 'time'
month = {}
monthLength = len(billMonth) - 1
for i in range(len(df)):
    temp = {}
    for j in range (0,monthLength,1):
        end = billMonth[j + 1] - offset
        temp[j] = df[i][df[i][timestamp_name].between(billMonth[j], end)]
    month[i] = temp
month

## Find energy values (kWh) and energy costs

In [ ]:
def energy(df, month, time_column = 'time', demand = 'Demand', cost = [False, False, False, False, False]): 
    # cost = [] vector must have five numeric elements/items, or be set to False if not 
    df = df.reset_index(drop=True)

    if len(month) != 0:
        monthLength = len(month) 
    else:
         monthLength = 0
            
    monthTitle = {}
    for i in range(0, monthLength, 1): # iterates through every billing month df
        monthTitle[i] = month[i][time_column].min().strftime('%b_%d_%Y') + ' to ' + month[i][time_column].max().strftime('%b_%d_%Y')

    columns = ['Billing Month', 'kiloWattHour Total', 'kiloWattHour Super Off-Peak', 'kiloWattHour Off-Peak', 'kiloWattHour Mid-Peak', 'kiloWattHour On-Peak']
    columns_cost = ['Total Energy Cost ($)', 'Super Off-Peak Energy Cost ($)', 'Off-Peak Energy Cost ($)', 'Mid-Peak Energy Cost ($)', 'On-Peak Energy Cost ($)']
    for j in range(5):
            if (type(cost[j]) == int or type(cost[j]) == float ):
                columns.append(columns_cost[j])
    
    col_len = int(len(columns))
    kiloWattHour = pd.DataFrame(np.zeros([monthLength, col_len])*np.nan, columns = columns)

    for i in range(0, monthLength, 1):
        month[i] = month[i].reset_index(drop=True)
        kiloWattHour['Billing Month'][i] = monthTitle[i]
        kiloWattHour['kiloWattHour Total'][i] = month[i][demand].sum() / 4
        kiloWattHour['kiloWattHour Super Off-Peak'][i] = month[i][demand][month[i].Peak == 'Super Off-Peak'].sum() / 4
        kiloWattHour['kiloWattHour Off-Peak'][i] = month[i][demand][month[i].Peak == 'Off-Peak'].sum() / 4
        kiloWattHour['kiloWattHour Mid-Peak'][i] = month[i][demand][month[i].Peak == 'Mid-Peak'].sum() / 4
        kiloWattHour['kiloWattHour On-Peak'][i] = month[i][demand][month[i].Peak == 'On-Peak'].sum() / 4
        
        for k in range(5):
            if (type(cost[k]) == int or type(cost[k]) == float ):
                kiloWattHour[columns_cost[k]][i] = kiloWattHour[columns[k + 1]][i]  *  cost[k]
                

    kiloWattHour.loc[len(kiloWattHour)] = np.nan
    kiloWattHour = kiloWattHour.shift()
    kiloWattHour['Billing Month'][0] = 'Entire Interval'
    kiloWattHour['kiloWattHour Total'][0] = df[demand].sum() / 4
    kiloWattHour['kiloWattHour Super Off-Peak'][0] = df[demand][df.Peak == 'Super Off-Peak'].sum() / 4
    kiloWattHour['kiloWattHour Off-Peak'][0] = df[demand][df.Peak == 'Off-Peak'].sum() / 4
    kiloWattHour['kiloWattHour Mid-Peak'][0] = df[demand][df.Peak == 'Mid-Peak'].sum() / 4
    kiloWattHour['kiloWattHour On-Peak'][0] = df[demand][df.Peak == 'On-Peak'].sum() / 4
    for l in range(0, 5, 1):
        
        if (type(cost[l]) == int or type(cost[l]) == float ):
            kiloWattHour[columns_cost[l]][0] = kiloWattHour[columns[l + 1]][0] *  cost[l]
                

    return kiloWattHour 

In [ ]:
sav = pd.DataFrame(np.zeros((8,5), dtype=int), columns = ['Scenario', 'Demand Charges ($)',  'Energy Charges ($)', 'Total Cost ($)', 'Emissions']) 
sav['Scenario'] = np.array([1,2,3,4,5,6,7,8])
sav

In [ ]:
cost = [[-0.1242, False, False, False, False], [-0.1242, False, False, False, False], [-0.1242, False, False, False, False], [-0.1242, False, False, False, False], [-0.1242, False, False, False, False], [-0.1242, False, False, False, False], [-0.1242, False, False, False, False], [-0.1242, False, False, False, False]]
kiloWattHour_load = {}
for i in range(len(df)):
    kiloWattHour_load[i] = energy(df[i],month[i], time_column = 'time', demand = 'grid.sou.S[1]', cost = cost[i])

In [ ]:
kiloWattHour_load

In [ ]:
energy_cost = []
for i in range(8):
    energy_cost.append(kiloWattHour_load[i][1:]['Total Energy Cost ($)'].sum())
#for i in range(3,len(df)):
#    energy_cost.append(kiloWattHour_load[i][1:]['Off-Peak Energy Cost ($)'].sum() + kiloWattHour_load[i][1:]['Mid-Peak Energy Cost ($)'].sum() + kiloWattHour_load[i][1:]['On-Peak Energy Cost ($)'].sum())

sav['Energy Charges ($)'] = np.array(energy_cost).astype(int)
sav

## Find power values (kW) and demand costs

In [ ]:
def max_peak(df, month, time_column = 'time', demand = 'Demand', cost = [False, False, False, False, False]):
    # cost = [] vector must have five numeric elements/items, or be set to False if not 
    df = df.reset_index(drop=True)

    if len(month) != 0:
        monthLength = len(month) 
    else:
         monthLength = 0

    monthTitle = {}
    for i in range(0, monthLength, 1): # iterates through every billing month df
        monthTitle[i] = month[i][time_column].min().strftime('%b_%d_%Y') + ' to ' + month[i][time_column].max().strftime('%b_%d_%Y')

    columns = ['Billing Month', 'max_peak Total', 'max_peak Super Off-Peak', 'max_peak Off-Peak', 'max_peak Mid-Peak', 'max_peak On-Peak']
    columns_cost = ['Total Demand Cost ($)', 'Super Off-Peak Demand Cost ($)', 'Off-Peak Demand Cost ($)', 'Mid-Peak Demand Cost ($)', 'On-Peak Demand Cost ($)']
    for j in range(5):
            if (type(cost[j]) == int or type(cost[j]) == float ):
                columns.append(columns_cost[j])
    col_len = int(len(columns))
    max_peak = pd.DataFrame(np.zeros([monthLength, col_len])*np.nan, columns = columns)
    for i in range(0, monthLength, 1):
        month[i] = month[i].reset_index(drop=True)
        max_peak['Billing Month'][i] = monthTitle[i]
        max_peak['max_peak Total'][i] = month[i][demand].min()
        max_peak['max_peak Super Off-Peak'][i] = month[i][demand][month[i].Peak == 'Super Off-Peak'].min()
        max_peak['max_peak Off-Peak'][i] = month[i][demand][month[i].Peak == 'Off-Peak'].min()
        max_peak['max_peak Mid-Peak'][i] = month[i][demand][month[i].Peak == 'Mid-Peak'].min()
        max_peak['max_peak On-Peak'][i] = month[i][demand][month[i].Peak == 'On-Peak'].min()
        for k in range(5):
            if (type(cost[k]) == int or type(cost[k]) == float ):
                max_peak[columns_cost[k]][i] = max_peak[columns[k + 1]][i]  *  cost[k]
                
    max_peak.loc[len(max_peak)] = np.nan
    max_peak = max_peak.shift()
    max_peak['Billing Month'][0] = 'Entire Interval'
    max_peak['max_peak Total'][0] = df[demand].min()
    max_peak['max_peak Super Off-Peak'][0] = df[demand][df.Peak == 'Super Off-Peak'].min()
    max_peak['max_peak Off-Peak'][0] = df[demand][df.Peak == 'Off-Peak'].min()
    max_peak['max_peak Mid-Peak'][0] = df[demand][df.Peak == 'Mid-Peak'].min()
    max_peak['max_peak On-Peak'][0] = df[demand][df.Peak == 'On-Peak'].min()
    
    for l in range(0, 5, 1):    
        if (type(cost[l]) == int or type(cost[l]) == float ):
            max_peak[columns_cost[l]][0] = max_peak[columns[l + 1]][0] *  cost[l]

    return max_peak

In [ ]:
cost = [[-12.48, False, False, False, False], [-12.48, False, False, False, False], [-12.48, False, False, False, False], [-12.48, False, False, False, False], [-12.48, False, False, False, False], [-12.48, False, False, False, False], [-12.48, False, False, False, False], [-12.48, False, False, False, False]]
max_peak_load = {}
for i in range(len(df)):
    max_peak_load[i] = max_peak(df[i],month[i],time_column = 'time', demand = 'grid.sou.S[1]', cost = cost[i])
max_peak_load

In [ ]:
demand_cost = []
for i in range(8):
    demand_cost.append(max_peak_load[i][1:]['Total Demand Cost ($)'].sum())
#for i in range(3,len(df)):
#    demand_cost.append(max_peak_load[i][1:]['Off-Peak Demand Cost ($)'].sum() + max_peak_load[i][1:]['Mid-Peak Demand Cost ($)'].sum() + max_peak_load[i][1:]['On-Peak Demand Cost ($)'].sum())

sav['Demand Charges ($)'] = np.array(demand_cost).astype(int)
sav

In [ ]:
sav['Energy Charges ($)'][2] = 0
sav

In [ ]:
sav['Total Cost ($)'] = sav['Demand Charges ($)'] + sav['Energy Charges ($)']
sav

## Emissions output

In [ ]:
er = pd.read_csv('/home/lencon/Git_Research/FISTS_2024/Code/Microgrid_Data/CAISOdata.csv')
er['emissons_rate'] = er[['imports_co2', 'natgas_co2', 'biogas_co2',
       'biomass_co2', 'geothermal_co2', 'coal_co2']].sum(axis=1) / er[['wind_curtail_MW', 'solar_curtail_MW', 'solar_MW', 'wind_MW',
       'geothermal_MW', 'biomass_MW', 'biogas_MW', 'sm_hydro_MW', 'battery_MW',
       'renewable_MW', 'natgas_MW', 'lg_hydro_MW', 'imports_MW', 'nuclear_MW',
       'coal_MW']].sum(axis=1)
er.date = pd.to_datetime(er.date.astype(str) + ' ' + er['5min_ending'].astype(str))
er['date'] = er['date'] + pd.DateOffset(years = 3)
er

In [ ]:
ers = resample(er, time_column = 'date')
ers = ers[ers.date.between('03/01/2022', '03/01/2023', inclusive = 'left')].reset_index(drop=True)
ers

In [ ]:
def get_emission(df, er):
    df['grid.sou.S[1]'][df['grid.sou.S[1]'] > 0] = 0
    e = df['grid.sou.S[1]'] * er.emissons_rate * -0.00025
    e = e.sum()
    return e
emissions = []
for i in range(len(df)):
    emissions.append(get_emission(df[i], ers))
sav['Emissions'] = np.array(emissions).astype(int)
sav

In [ ]:
sav.to_latex('/home/lencon/Git_Research/FISTS_2024/Conference_Paper/LaTeX/Table/kw_kwh_co2_run_4.tex', index = False)

In [ ]:
def day_time(freq = '1min'):
    # Create the number on intervals during the day
    time_array = pd.date_range('0:00', '23:59:59', freq=freq).strftime('%H:%M')
    return(time_array)

def time_sep_label(df, time_column = 'time', resample_rate = '1T', value_column = 'value'):
    # Ensure time column is in datetime format
    df[time_column] = pd.to_datetime(df[time_column])
    # Set time column as index
    df = df.set_index(df[time_column])
    # Remove time column from dataframe since it is now the index
    df = df.drop(columns = [time_column])
    # Clean up and resample missing data 
    df = df.resample(resample_rate).mean()
    # Extract the time period ths user wants in a day as an integer
    resample_rate_int =  int(resample_rate.rstrip(resample_rate[-1]))
    # Change '1T' to '1min' Call the daytime function to produce the time intervals
    time_array = day_time(str(resample_rate_int) + 'min')
    # Create a new column for labeling each row 
    df['time_interval_label'] = np.nan
    # Declare the list for indexes for ever interval
    avg_day = []
    # Extract the indexes for ever interval
    for i in range(len(time_array) - 1):
        avg_day.append(df[value_column].between_time(time_array[i], time_array[i + 1], inclusive = 'left').mean())
    return[time_array, avg_day]

In [ ]:
def get_emission(df, er):
    df['grid.sou.S[1]'][df['grid.sou.S[1]'] > 0] = 0
    e = df['grid.sou.S[1]'] * er.emissons_rate * -0.00025
    return e
edf = pd.DataFrame({'time' : df[i].time})
for i in range(len(df)):
    edf[str(i)] = (get_emission(df[i], ers))
edf

In [ ]:
avg_emissions = pd.DataFrame()
for i in range(len(df)):
    time_array, avg_day = time_sep_label(edf, resample_rate = '60T', value_column = str(i))
    avg_emissions[str(i)] = avg_day
avg_emissions

In [ ]:
sav['% Emissions Savings'] = (sav['Emissions'].max() - sav['Emissions']) / sav['Emissions'].max() * 100
sav['% Emissions Savings'] = sav['% Emissions Savings'].astype(int)
sav

## Plot Data 

In [ ]:
import os
os.chdir('/home/lencon/Git_Research/FISTS_2024/Conference_Paper/LaTeX/Fig')

In [ ]:
def plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color, fig_show = False):
    # Define the figure
    fig = go.Figure()
    # Add data to figure
    for i in range(0, len(x_data), 1):
        fig.add_trace(go.Scattergl(x=x_data[i], y=y_data[i], opacity=.5, name= line_name[i], line_color = line_color[i]))
            
    # Create Layout for the Html
    
    fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=True
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=y_title,
            )
        ),
        plot_bgcolor = "white"
    )
    
    fig.write_html((filename + '.html'), include_mathjax='cdn')
    fig = go.Figure()
    # Add data to figure
    for i in range(0, len(x_data), 1):
        fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], opacity=.5, name= line_name[i], line_color = line_color[i]))
    fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=False
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=y_title,
            )
        ),
        plot_bgcolor = "white"
    )
    fig.write_image(filename + '.svg')
    fig.write_image(filename + '.pdf')
    # fig.write_image(filename + '.png') Do not use png in your code this is only an example to compare png to vector images
    if fig_show:
        fig.show()

In [ ]:
def box_plot_no_scatter(y_data, name, title, x_title, y_title, filename, color_bool = False, color = [], fig_show = False):
    fig = go.Figure()
    for i in range(0, len(y_data), 1):
        fig.add_trace(go.Box(
            y = y_data[i],
            name=name[i],
            boxpoints=False, # no data points
#             if color_bool:
#                 marker_color=color[i],
#                 line_color=color[i]
        ))
        for x in zip(["min","q1","med","q3","max"],y_data[i].quantile([0,0.25,0.5,0.75,1])):
            if x[0] == "q1":
                y_loc = y_data[i].min() + 0.25*(y_data[i].max() - y_data[i].min())
            elif x[0] == "med":
                y_loc = y_data[i].min() + 0.5*(y_data[i].max() - y_data[i].min())
            elif x[0] == "q3":
                y_loc = y_data[i].min() + 0.75*(y_data[i].max() - y_data[i].min())
            else: 
                y_loc = x[1]
            fig.add_annotation(
                x=i + 0.5,
                y=y_loc,
                text=x[0] + ":" + str(np.round(x[1], 1)),
                showarrow=False,
                font=dict(
                    size=8,
                    color="black"
                    ),
                align="center",
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="#636363",
                ax=20,
                ay=-30,
                bordercolor="#c7c7c7",
                borderwidth=1,
                borderpad=4,
                bgcolor='deepskyblue',
                opacity=0.9
                )
    fig.update_layout(
        xaxis_title=x_title,
        yaxis_title=y_title,
        title_text=title,
#         legend = dict(y=-0.5),
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
#        plot_bgcolor = "white"
    )
    fig.write_html((filename + '.html'), include_mathjax='cdn')
    fig.write_image(filename + '.svg')
    fig.write_image(filename + '.pdf')
    if fig_show:
        fig.show()
    return

In [ ]:
y_data[0].quantile([0])

In [ ]:
for x in zip(["min","q1","med","q3","max"],y_data[i].quantile([0,0.25,0.5,0.75,1])):
    print(x[1])

In [ ]:
#create title for plots
#create filename for plots
site_name = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"]
plot_type = "Microgrid Output"
month_title = {}
for i in range(len(df)):
    temp = {}
    for j in range (0,len(month[i])):
        temp[j] = site_name[i] + "<br>" + plot_type + "<br>From: " +  month[i][j].time.min().strftime('%B %d, %Y') + " to " + month[i][j].time.max().strftime('%B %d, %Y') 
    month_title[i] = temp 
month_title

In [ ]:
#create filename for plots
site_name = ["Scenario_1_Run_3", "Scenario_2_Run_3", "Scenario_3_Run_3", "Scenario_4_Run_3", "Scenario_5_Run_3", "Scenario_6_Run_3", "Scenario_7_Run_3", "Scenario_8_Run_3"]
plot_type = "Mg_Output"
filename_list = {}
for i in range(len(df)):
    temp = {}
    for j in range (0,len(month[i])):
        temp[j] = str(j) + '_' + site_name[i] + '_' + plot_type + '_' + month[i][j].time.min().strftime('%b_%d_%Y') + "_to_" + month[i][j].time.max().strftime('%b_%d_%Y') 
    filename_list[i] = temp
filename_list

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["BESS", "Solar", "Grid Meter (Net Load)"]
line_color = ["deepskyblue", "red", "green"]
for i in range (0,len(df)):
    for j in range(0, len(month[i])):
        x_data = [month[i][j].time] 
        y_data = [month[i][j]["grid.sou.S[1]"]]
        title = month_title[i][j]
        filename = filename_list[i][j]
        plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=4, cols=2,
    subplot_titles=("Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"))

# fig.add_trace(go.Scatter(x=month[0][4].time , y=month[0][4]["BESS.P"]),
#               row=1, col=1)

# fig.add_trace(go.Scatter(x=month[1][4].time , y=month[1][4]["BESS.P"]),
#               row=1, col=2)
fig.add_trace(go.Scatter(x=month[0][4].time , y=month[0][4]["grid.sou.S[1]"], opacity=.5, name = "Grid Meter (Net Load)", legendgroup = '0'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=month[1][4].time , y=month[1][4]["grid.sou.S[1]"], opacity=.5, name = "Grid Meter (Net Load)", legendgroup = '1'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=month[2][4].time , y=month[2][4]["grid.sou.S[1]"], opacity=.5, name = "Grid Meter (Net Load)", legendgroup = '2'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=month[3][4].time , y=month[3][4]["grid.sou.S[1]"], opacity=.5, name = "Grid Meter (Net Load)", legendgroup = '3'),
              row=2, col=2)

fig.add_trace(go.Scatter(x=month[4][4].time , y=month[4][4]["grid.sou.S[1]"], opacity=.5, name = "Grid Meter (Net Load)", legendgroup = '4'),
              row=3, col=1)

fig.add_trace(go.Scatter(x=month[5][4].time , y=month[5][4]["grid.sou.S[1]"], opacity=.5, name = "Grid Meter (Net Load)", legendgroup = '5'),
              row=3, col=2)
fig.add_trace(go.Scatter(x=month[6][4].time , y=month[6][4]["grid.sou.S[1]"], opacity=.5, name = "Grid Meter (Net Load)", legendgroup = '6'),
              row=4, col=1)

fig.add_trace(go.Scatter(x=month[7][4].time , y=month[7][4]["grid.sou.S[1]"], opacity=.5, name = "Grid Meter (Net Load)", legendgroup = '7'),
              row=4, col=2)
#################
fig.add_trace(go.Scatter(x=month[0][4].time , y=month[0][4]["generator.P"], opacity=.5, name = "Solar", legendgroup = '0'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=month[1][4].time , y=month[1][4]["generator.P"], opacity=.5, name = "Solar", legendgroup = '1'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=month[2][4].time , y=month[2][4]["generator.P"], opacity=.5, name = "Solar", legendgroup = '2'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=month[3][4].time , y=month[3][4]["generator.P"], opacity=.5, name = "Solar", legendgroup = '3'),
              row=2, col=2)

fig.add_trace(go.Scatter(x=month[4][4].time , y=month[4][4]["generator.P"], opacity=.5, name = "Solar", legendgroup = '4'),
              row=3, col=1)

fig.add_trace(go.Scatter(x=month[5][4].time , y=month[5][4]["generator.P"], opacity=.5, name = "Solar", legendgroup = '5'),
              row=3, col=2)
fig.add_trace(go.Scatter(x=month[6][4].time , y=month[6][4]["generator.P"], opacity=.5, name = "Solar", legendgroup = '6'),
              row=4, col=1)

fig.add_trace(go.Scatter(x=month[7][4].time , y=month[7][4]["generator.P"], opacity=.5, name = "Solar", legendgroup = '7'),
              row=4, col=2)
###############
fig.add_trace(go.Scatter(x=month[1][4].time , y=month[1][4]["BESS.P"], opacity=.5, name = "BESS", legendgroup = '1'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=month[2][4].time , y=month[2][4]["BESS.P"], opacity=.5, name = "BESS", legendgroup = '2'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=month[3][4].time , y=month[3][4]["BESS.P"], opacity=.5, name = "BESS", legendgroup = '3'),
              row=2, col=2)

fig.add_trace(go.Scatter(x=month[5][4].time , y=month[5][4]["BESS.P"], opacity=.5, name = "BESS", legendgroup = '5'),
              
              row=3, col=2)
fig.add_trace(go.Scatter(x=month[6][4].time , y=month[6][4]["BESS.P"], opacity=.5, name = "BESS", legendgroup = '6'),
              row=4, col=1)

fig.add_trace(go.Scatter(x=month[7][4].time , y=month[7][4]["BESS.P"], opacity=.5, name = "BESS", legendgroup = '7'),
              row=4, col=2)
################


fig.update_layout(
                  title_text="Microgrid Power Output from the openModelica Simulation")

fig.show()
filename = "scenario_subplot"
fig.write_image(filename + '.svg')
fig.write_image(filename + '.pdf')

In [ ]:
#create title for plots
#create filename for plots
site_name = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"]
plot_type = "Microgrid Output"
ei_title = {}
for i in range(len(df)):
        ei_title[i] = site_name[i] + "<br>" + plot_type + "<br>From: " +  df[i].time.min().strftime('%B %d, %Y') + " to " + df[i].time.max().strftime('%B %d, %Y')     
ei_title

In [ ]:
#create filename for plots
site_name = ["Scenario_1_Run_3", "Scenario_2_Run_3", "Scenario_3_Run_3", "Scenario_4_Run_3", "Scenario_5_Run_3", "Scenario_6_Run_3", "Scenario_7_Run_3", "Scenario_8_Run_3"]
plot_type = "Mg_Output_Run_3"
ei_filename_list = {}
for i in range(len(df)):
        ei_filename_list[i] = str(i) + '_' + plot_type + '_' + df[i].time.min().strftime('%b_%d_%Y') + "_to_" + df[i].time.max().strftime('%b_%d_%Y') 
ei_filename_list

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["BESS", "Solar", "Grid Meter"]
line_color = ["deepskyblue", "red", "green"]
title = ei_title
filename = ei_filename_list
for i in range(len(df)):
    x_data = [df[i]['time']] 
    y_data = [df[i]["grid.sou.S[1]"]]
    plot_graph(x_data, y_data, line_name, title[i], x_title, y_title, filename[i], line_color)

In [ ]:
len(month[0])

In [ ]:
#create title for plots
#create filename for plots
site_name = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"]
plot_type = "Microgrid Output"
bx_title = {}
for i in range(len(df)):
        bx_title[i] = site_name[i] + "<br>" + plot_type
bx_title

In [ ]:
#create filename for plots
site_name = ["Scenario_1", "Scenario_2", "Scenario_3", "Scenario 4", "Scenario 6", "Scenario 7", "Scenario 8"]
plot_type = "Mg_Output_Run_3"
bx_filename_list = {}
for i in range(len(df)):
        bx_filename_list[i] = str(i) + '_' + plot_type +"_bx_pwr"
bx_filename_list

In [ ]:
name = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"]
x_title = "Scenarios"
y_title = "Demand (kW)"
y_data = []
for i in range(0, len(df)):
    #x_data.append(df[i].time) 
    y_data.append(df[i]["grid.sou.S[1]"])
title = "Scenarios Grid Power Output"
filename = "scenarios_power_output_boxplot"
box_plot_no_scatter(y_data, name, title, x_title, y_title, filename, fig_show = True)

In [ ]:
#create title for plots
#create filename for plots
#site_name = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"]
plot_type = "Scenarios Power Output"
month_title = {}
for i in range(1):
    temp = {}
    for j in range (0,len(month[i])):
        temp[j] = plot_type + "<br>From: " +  month[i][j].time.min().strftime('%B %d, %Y') + " to " + month[i][j].time.max().strftime('%B %d, %Y') 
    month_title = temp 
month_title

In [ ]:
#create filename for plots
site_name = ["Scenario_1", "Scenario_2", "Scenario_3", "Scenario_4", "Scenario_5", "Scenario_6", "Scenario_7", "Scenario_8"]
plot_type = "Scn_Output_Run_3"
filename_list = {}
for i in range(1):
    temp = {}
    for j in range (0,len(month[i])):
        temp[j] = str(j) + '_' + plot_type + '_' + month[i][j].time.min().strftime('%b_%d_%Y') + "_to_" + month[i][j].time.max().strftime('%b_%d_%Y') 
    filename_list = temp
filename_list

In [ ]:
name = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"]
x_title = "Scenarios"
y_title = "Demand (kW)"
for j in range(0, len(month_title)):
    y_data = []
    for i in range(0, len(month)):
    #x_data.append(df[i].time) 
        y_data.append(month[i][j]["grid.sou.S[1]"])
    title =  month_title[j]
    filename = filename_list[j]
    box_plot_no_scatter(y_data, name, title, x_title, y_title, filename, fig_show = True)

In [ ]:
name = ["Scenario 2", "Scenario 3", "Scenario 4", "Scenario 6", "Scenario 7", "Scenario 8"]
x_title = "Scenarios"
y_title = "Demand (kW)"
y_data = []
for i in [1, 2, 3, 5, 6, 7]:
    #x_data.append(df[i].time) 
    y_data.append(df[i]["ev_charger_level_2_load.P"] + df[i]["ev_charger_level_3_load.P"])
title = "Scenarios EV Power Output"
filename = "scenarios_ev_power_output_boxplot"
box_plot_no_scatter(y_data, name, title, x_title, y_title, filename, fig_show = True)

In [ ]:
name = ["Scenario 1","Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"]
x_title = "Scenarios"
y_title = "Demand (kW)"
y_data = []
for i in [0, 1, 2, 3, 4, 5, 6, 7]:
    #x_data.append(df[i].time) 
    y_data.append(df[i]["ev_charger_level_2_load.P"] + df[i]["ev_charger_level_3_load.P"] + df[i]["building_load.P"] + df[i]["plug_load_resitive_load.P"]) 
title = "Scenarios Microgrid Load"
filename = "scenarios_mg_load_output_boxplot"
box_plot_no_scatter(y_data, name, title, x_title, y_title, filename, fig_show = True)

In [ ]:
name = ["Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"]
x_title = "Scenarios"
y_title = "Demand (kW)"
y_data = []
for i in range(4, len(df)):
    #x_data.append(df[i].time) 
    y_data.append(df[i]["BESS.P"])
title = "BESS Output"
filename = "bess_boxplot"
box_plot_no_scatter(y_data, name, title, x_title, y_title, filename, fig_show = True)

In [ ]:
df[0]["grid.sou.S[1]"]

In [ ]:
month[0]

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Scenario 1", "Scenario 2", "Scenario 3"]
line_color = ["deepskyblue", "red", "orange"]
title = 'Scenario Comparison<br>Grid Meter<br>From: March 01, 2022 to February 28, 2023'
filename = 'net_load_scenario_comparison_run_2'
x_data = []
y_data = []
for i in range(len(df)):
    x_data.append(df[i]['time'])
    y_data.append(df[i]["grid.sou.S[1]"])
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Scenario 1", "Scenario 2", "Scenario 6", "Scenario 7"]
line_color = ["deepskyblue", "red", "orange", "magenta"]
title = 'Scenario Comparison<br>Grid Meter<br>From: July 01, 2022 to August 01, 2022'
filename = 'net_load_scenario_comparison_summer_run_3'
x_data = []
y_data = []
for i in [0,1,5,6]:
    x_data.append(df[i]['time'][df[i]['time'].between('07-01-2022', '08-01-2022')])
    y_data.append(df[i]["grid.sou.S[1]"][df[i]['time'].between('07-01-2022', '08-01-2022')])
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Scenario 6"]
line_color = ["deepskyblue"]
title = 'Scenario 6 Peak Shaving<br>Grid Meter<br>From: July 01, 2022 to August 01, 2022'
filename = 'scenario_6_peak_shaving'
x_data = []
y_data = []
i = 5
x_data.append(df[i]['time'][df[i]['time'].between('07-01-2022', '08-01-2022')])
y_data.append(df[i]["grid.sou.S[1]"][df[i]['time'].between('07-01-2022', '08-01-2022')])
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Scenario 7"]
line_color = ["deepskyblue"]
title = 'Scenario 7 Peak Shaving<br>Grid Meter<br>From: July 01, 2022 to August 01, 2022'
filename = 'scenario_7_peak_shaving'
x_data = []
y_data = []
i = 6
x_data.append(df[i]['time'][df[i]['time'].between('07-01-2022', '08-01-2022')])
y_data.append(df[i]["grid.sou.S[1]"][df[i]['time'].between('07-01-2022', '08-01-2022')])
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
df[2]["grid.sou.S[1]"].describe()

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Level 2"]
line_color = ["deepskyblue"]
title = 'Level 2 Chargers<br>Power Output<br>From: March 01, 2022 to February 28, 2023'
filename = 'ev_l2_po'
x_data = [df[0]['time']]
y_data = [df[0]['ev_charger_level_2_load.P']]
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Level 3"]
line_color = ["deepskyblue"]
title = 'Level 3 Chargers<br>Power Output<br>From: March 01, 2022 to February 28, 2023'
filename = 'ev_l3_po'
x_data = [df[0]['time']]
y_data = [df[0]['ev_charger_level_3_load.P']]
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
sav['Total Cost ($)'][3:]

In [ ]:
[100, 250, 500, 1000, 2000] * 2

In [ ]:
x_title = "BESS Capacity"
y_title = "Cost ($)"
line_name = ["Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"]
line_color = ["deepskyblue", "red", "orange", "magenta", "teal"]
title = 'BESS Capacity Comparison'
x_data = [[100, 250, 500, 1000, 2000], [100, 250, 500, 1000, 2000]] 
y_data = [sav['Total Cost ($)'][3:], sav['Emissions'][3:]]
filename = 'bess_capacity_comparison'
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
sav.columns

In [ ]:
x_title = "BESS Capacity"
y1_title = "Cost ($)"
y2_title = "$CO_{2} \ Emissions \ (_{m}Ton)$"
title = 'BESS Capacity Comparison'
filename = 'bess_capacity_comparison'
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=[100, 250, 500, 1000, 2000] , y=sav['Total Cost ($)'][3:], line_color = "deepskyblue", name="Total Cost ($)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=[100, 250, 500, 1000, 2000] , y=sav['Emissions'][3:], line_color =  "red", name=r"Emissions $_{m}Ton_{CO_{2}}$"),
    secondary_y=True,
)
fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=False
            )
        ),
        plot_bgcolor = "white"
    )
fig.update_yaxes(title_text=y1_title, secondary_y=False)
fig.update_yaxes(title_text=y2_title, secondary_y=True)
fig.write_html(filename + '.html')
fig.write_image(filename + '.svg')
fig.write_image(filename + '.pdf')

In [ ]:
x_title = "BESS Capacity"
y1_title = "Cost ($)"
y2_title = "$CO_{2} \ Emissions \ (_{m}Ton)$"
title = 'BESS Capacity Comparison'
filename = 'bess_capacity_comparison'
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=[100, 250, 500, 1000, 2000] , y=sav['Total Cost ($)'][3:], line_color = "deepskyblue", name="Total Cost ($)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=[100, 250, 500, 1000, 2000] , y=sav['Emissions'][3:], line_color =  "red", name=r"Emissions $_{m}Ton_{CO_{2}}$"),
    secondary_y=True,
)
fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=False
            )
        ),
        plot_bgcolor = "white"
    )
fig.update_yaxes(title_text=y1_title, secondary_y=False)
fig.update_yaxes(title_text=y2_title, secondary_y=True)
fig.write_html(filename + '.html')
fig.write_image(filename + '.svg')
fig.write_image(filename + '.pdf')

In [ ]:
x_title = "BESS Capacity"
y1_title = "Cost ($)"
y2_title = "$CO_{2} \ Emissions \ (_{m}Ton)$"
title = 'BESS Capacity Comparison'
filename = 'bess_capacity_comparison'
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=[100, 250, 500, 1000, 2000] , y=sav['Total Cost ($)'][3:], line_color = "deepskyblue", name="Total Cost ($)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=[100, 250, 500, 1000, 2000] , y=sav['Emissions'][3:], line_color =  "red", name=r"Emissions $_{m}Ton_{CO_{2}}$"),
    secondary_y=True,
)
fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=False
            )
        ),
        plot_bgcolor = "white"
    )
fig.update_yaxes(title_text=y1_title, secondary_y=False)
fig.update_yaxes(title_text=y2_title, secondary_y=True)
fig.write_html(filename + '.html')
fig.write_image(filename + '.svg')
fig.write_image(filename + '.pdf')

In [ ]:
[sav['Emissions'][:3], sav['Emissions'][5]]

In [ ]:
sav['Emissions'][[0,3,4,5,6,7]]

In [ ]:
x_title = "BESS Capacity"
y1_title = "Cost ($)"
y2_title = "CO<sub>2</sub> Emissions (<sub>m</sub>Ton)"
title = 'BESS Capacity Comparison'
filename = 'bess_capacity_comparison'
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=[100, 250, 500, 1000, 2000] , y=sav['Demand Charges ($)'][[1,3,4,5,6,7]], line_color = "yellow", name="Demand Charges ($)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=[100, 250, 500, 1000, 2000] , y=sav['Energy Charges ($)'][[1,3,4,5,6,7]], line_color = "orange", name="Energy Charges ($)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=[0, 100, 250, 500, 1000, 2000] , y=sav['Total Cost ($)'][[1,3,4,5,6,7]], line_color = "deepskyblue", name="Total Cost ($)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=[0, 100, 250, 500, 1000, 2000] , y=sav['Emissions'][[1,3,4,5,6,7]], line_color =  "red", name=r"CO<sub>2</sub> Emissions (<sub>m</sub>Ton)"),
    secondary_y=True,
)
fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=False
            )
        ),
        plot_bgcolor = "white"
    )
fig.update_yaxes(title_text=y1_title, secondary_y=False)
fig.update_yaxes(title_text=y2_title, secondary_y=True)
fig.write_html(filename + '.html')
fig.write_image(filename + '.svg')
fig.write_image(filename + '.pdf')
fig.show()

In [ ]:
x_title = "Scenarios"
y1_title = "Cost ($)"
y2_title = "CO<sub>2</sub> Emissions (<sub>m</sub>Ton)"
title = 'EV Charging Scenarios Comparison'
filename = 'mg_scene_comparison'
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=['No EVCS, Solar, BESS', 'EVCS No Solar, BESS', 'BESS, Solar, No EVCS', 'EVCS, Solar, BESS'] , y=sav['Total Cost ($)'][[0,1,2,5]], line_color = "deepskyblue", name="Total Cost ($)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=['No EVCS, Solar, BESS', 'EVCS No Solar, BESS', 'BESS, Solar, No EVCS', 'EVCS, Solar, BESS'] , y=sav['Emissions'][[0,1,2,5]], line_color =  "red", name=r"CO<sub>2</sub> Emissions (<sub>m</sub>Ton)"),
    secondary_y=True,
)
fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=False
            )
        ),
        plot_bgcolor = "white"
    )
fig.update_yaxes(title_text=y1_title, secondary_y=False)
fig.update_yaxes(title_text=y2_title, secondary_y=True)
fig.write_html(filename + '.html')
fig.write_image(filename + '.svg')
fig.write_image(filename + '.pdf')
fig.show()

In [ ]:
import plotly.graph_objects as go
x_title = "Scenarios"
y1_title = r"Cost ($)"
y2_title = "CO<sub>2</sub> Emissions (<sub>m</sub>Ton)"
title = 'EV Charging Scenarios Comparison'
filename = 'mg_scene_comparison'
xs = ['No EVCS, Solar, BESS', 'EVCS No Solar, BESS', 'BESS, Solar, No EVCS', 'EVCS, Solar, BESS']
fig = go.Figure(data=[
    go.Bar(name=y1_title, x=xs, y=sav['Total Cost ($)'][[0,1,2,5]], yaxis='y', offsetgroup=1),
    go.Bar(name=y2_title, x=xs, y=sav['Emissions'][[0,1,2,5]], yaxis='y2', offsetgroup=2)
],
                layout={'yaxis': {'title': y1_title},
                        'yaxis2': {'title': y2_title, 'overlaying': 'y', 'side': 'right'}
                       })
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(
        title=go.layout.Title(
            text= title,
            xref="paper",
            x=0.5
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=x_title,
            ),
            rangeslider=dict(
                visible=False
            )
        ),
        plot_bgcolor = "white"
    )
fig.write_html(filename + '.html')
fig.write_image(filename + '.svg')
fig.write_image(filename + '.pdf')
fig.show()

In [ ]:
x_title = "Time (Hours of the Day)"
y_title = r"Emissions Rate (kg CO<sub>2</sub>)"
line_name = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5", "Scenario 6", "Scenario 7", "Scenario 8"]
line_color = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
] #["deepskyblue", "red", "green", "magenta"]
title = 'Scenario Comparison<br>Emissions Ouput<br>Average Day'
filename = 'emissions_scenario_comparison_run_3'
x_data = []
y_data = []
for i in range(len(df)):
    x_data.append(np.arange(24))
    y_data.append(avg_emissions[str(i)] * 1000)
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
edf

In [ ]:
df = pd.read_csv('/home/lencon/Git_Research/FISTS_2024/Code/Microgrid_Data/microgrid_data.csv')
df

In [ ]:
df['time'] = pd.to_datetime(df['time'] + 1646092800, unit = 's')
df

In [ ]:
df[df.select_dtypes(include=['number']).columns] /= 1000
df

In [ ]:
def first_resample(df, time_column = 'time', resample_rate = '15T'):
    df[time_column] = pd.to_datetime(df[time_column])
    df = df.set_index(df[time_column])
    df = df.drop(columns = [time_column])
    df = df.resample(resample_rate).mean()  #df.groupby(pd.Grouper(key=time_column, freq=resample_rate)).ffill().bfill() 
    df.insert(loc=0, column=time_column, value=df.index) #df.insert(loc=0, column=time_column, value=np.arange(0, len(df.index), 1, dtype=int)) #
    df = df.reset_index(drop=True)
    #df.fillna(0)
    #df['value'] = np.floor(pd.to_numeric(df['value'], errors='coerce')).astype('Int64')
    #df = df.fillna(0)
    return df

In [ ]:
df = first_resample(df)
df

In [ ]:
act = pd.read_csv('/home/lencon/Git_Research/FISTS_2024/Code/Microgrid_Data/optimization_1084_2022_2023.csv')
act

In [ ]:
act[act.select_dtypes(include=['number']).columns] /= 1000
act

In [ ]:
df['meter_load'] = act['meter_load']
df

In [ ]:
df.corr()

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Grid Meter (Real Data)", "Grid Meter (Modelica Simulation)"]
line_color = ["deepskyblue", "red"]
x_data = [act.time, df.time] 
y_data = [act.meter_load, df["grid.sou.S[1]"]]
site_name_t = "Modelica Validation"
plot_type_t = "15 Minute Data"
site_name_f = "ucr"
plot_type_f = "15_Minute_Data"
title = site_name_t + "<br>" + plot_type_t + "<br>From: " +  df.time.min().strftime('%B %d, %Y') + " to " + df.time.max().strftime('%B %d, %Y') 
filename = site_name_f + '_' + plot_type_f + '_' + df.time.min().strftime('%b_%d_%Y') + "_to_" + df.time.max().strftime('%b_%d_%Y') 
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)

In [ ]:
x_title = "Time"
y_title = "Demand (kW)"
line_name = ["Grid Meter (Real Data)", "Grid Meter (Modelica Simulation)"]
line_color = ["deepskyblue", "red"]
x_data = [act['time'][act['time'].between('06-01-2022', '07-01-2022')], df['time'][df['time'].between('06-01-2022', '07-01-2022')]] 
y_data = [act['meter_load'][act['time'].between('06-01-2022', '07-01-2022')], df["grid.sou.S[1]"][df['time'].between('06-01-2022', '07-01-2022')]]
site_name_t = "Modelica Validation"
plot_type_t = "15 Minute Data"
site_name_f = "ucr"
plot_type_f = "15_Minute_Data_Jun"
title = site_name_t + "<br>" + plot_type_t + "<br>From: " +  x_data[1].min().strftime('%B %d, %Y') + " to " + x_data[1].max().strftime('%B %d, %Y') 
filename = site_name_f + '_' + plot_type_f + '_' + x_data[1].min().strftime('%b_%d_%Y') + "_to_" + x_data[1].max().strftime('%b_%d_%Y') 
plot_graph(x_data, y_data, line_name, title, x_title, y_title, filename, line_color)